In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import wxconv
import glob
from wxconv import WXC
import codecs
import os,subprocess

In [2]:
pwd()


'/Users/adityaparashar/LTRC'

In [3]:
con = WXC(order='utf2wx', lang='hin')
con.convert('क्या कौन कहाँ कैसे क्यों कब कौन सा किसे, किससे, किसको किसका,किसकी किसलिए कितना')

'kyA kOna kahAz kEse kyoM kaba kOna sA kise, kisase, kisako kisakA,kisakI kisalie kiwanA'

In [5]:
%cd E:\\META\\ltrc

E:\META\ltrc


In [3]:
# function to check for different verb signifiers in the dictionary
verb_s = ['A.','U.','P.','Ā.']
def check_verb_pos(s):
    for i in verb_s:
        if i in s:
            return True
    return False

In [14]:
hws = []
b1s = []
# function to parse the html data from the apte dictionary
def get_word_meaning(results,d,ind):
    for div in results[1:]:
        hw = div.find('hw')
        pos = ''
        if hw:
            try:
                #print([hw.next_sibling,hw.next_sibling.next_sibling])
                b1 = hw.find_all('b')[0]
                word = b1.text
                words = []
                if len(hw.find_all('b')) >=2:
                    ix = 0
                    while ix < len(hw.find_all('b')):
                        words.append(hw.find_all('b')[ix].text)
                        ix+=2
                if check_verb_pos(str(hw.next_sibling)):
                    pos = 'verb'
                if '<b>' in str(hw.next_sibling.next_sibling):
                    if check_verb_pos(hw.find_next_sibling('b').next_sibling):
                        pos = 'verb'
                for word in words:
                #hw.find_next_siblings('i')

                    if hw.next_sibling.strip(' ') == '[':
                        #print('ppp')
                        temp = hw.next_sibling
                        while ']' not in temp:
                            #print([temp])
                            temp = temp.next_sibling
                        if 'To ' in temp:
                            pos = 'verb'
                        d['word'].append(word)
                        d['meaning'].append(temp)
                        d['pos'].append(pos)
                        #print(['k',temp])
                    elif hw.next_sibling == ' ':
                        if '<i>' in str(hw.next_sibling.next_sibling):
                            pos = hw.find_next('i').text
                            if hw.find_next('i').next_sibling.strip(' ') == '[':
                                temp = hw.next_sibling

                                while ']' not in temp:
                                    #print([temp])
                                    temp = temp.next_sibling
                                d['word'].append(word)
                                d['meaning'].append(temp)
                                d['pos'].append(pos)
                            else:
                                m = hw.find_next('i').next_sibling
                                d['word'].append(word)
                                d['meaning'].append(m)
                                d['pos'].append(pos)
                        else:
                            m = hw.find_next('i').next_sibling
                            d['word'].append(word)
                            d['meaning'].append(m)
                            d['pos'].append(pos)
                    if pos == '':
                        pos = 'noun'


                    bs = hw.find_next_siblings('b')

                    if len(bs) >0:
                        word_b = word
                        for b in bs:
                            if b.text.strip('-').isdigit():
                                bnext = b.next_sibling
                                d['word'].append(word_b)
                                d['meaning'].append(bnext)
                                d['pos'].append(pos)
                            elif b.find('d'):
                                d_s = b.find_all('d')
                                word_bs = []
                                for d1 in d_s:
                                    word_bs.append( word+ ' '+d1.text)
                                word_b =  word+ ' '+b.find('d').text
                                temp= b.next_sibling
                                if temp.strip(' ') == '[':
                                    while ']' not in temp:
                                        #print([temp])
                                        temp = temp.next_sibling
                                for w in word_bs:
                                    d['word'].append(w)
                                    d['meaning'].append(temp)
                                    d['pos'].append(pos)
                                if temp ==' ':
                                    if  '<i>' in str(temp.next_sibling):
                                        temp2 = temp.next_sibling.next_sibling
                                        if ' [' in temp2:
                                            while ']' not in temp2:
                                                temp2 = temp2.next_sibling
                                        pos = temp.find_next('i').text
                                        for w in word_bs:
                                            d['word'].append(w)
                                            d['meaning'].append(temp2)
                                            d['pos'].append(pos)

            except:
                print([ind,hw])

                    
                    
        

In [15]:
# parsing the 1768 pages of the Apte dictionary
d = {'word':[],'meaning':[],'pos':[]}
for i in range(1769):
    URL = "https://dsal.uchicago.edu/cgi-bin/app/apte_query.py?page=" + str(i)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id = "results_display")
    results  = results.find_all("div")
    get_word_meaning(results,d,i)

[23, <hw><d><b>अङकतिः</b> <b>aṅakatiḥ</b></d></hw>]
[38, <hw><d><b>अतप्त</b> <b>atapta</b></d></hw>]
[53, <hw><d><b>अदर्शः</b> <b>adarśaḥ</b></d></hw>]
[63, <hw><d><b>अधिजिह्वः</b> <b>adhijihvaḥ</b></d></hw>]
[66, <hw><d><b>अधिवेत्तृ</b> <b>adhivettṛ</b></d></hw>]
[75, <hw><d><b>अनपकरणम्</b> <b>anapakaraṇam</b>, <b>अनपकर्मन्</b> <b>anapakarman</b>, <b>अनपक्रिया</b> <b>anapakriyā</b></d></hw>]
[97, <hw><d><b>अनुपन्यासः</b> <b>anupanyāsaḥ</b></d></hw>]
[102, <hw><d><b>अनुमननम्</b> <b>anumananam</b></d></hw>]
[106, <hw><d><b>अनुवक्र</b> <b>anuvakra</b></d></hw>]
[106, <hw><d><b>अनुवचनम्</b> <b>anuvacanam</b></d></hw>]
[127, <hw><d><b>अन्दिका</b> <b>andikā</b></d></hw>]
[143, <hw><d><b>अपयानम्</b> <b>apayānam</b></d></hw>]
[148, <hw><d><b>अपवर्तः</b> <b>apavartaḥ</b></d></hw>]
[154, <hw><d><b>अपासनम्</b> <b>apāsanam</b></d></hw>]
[176, <hw><d><b>अभिपतनम्</b> <b>abhipatanam</b></d></hw>]
[180, <hw><d><b>अभिमर्शः</b> <b>abhimarśaḥ</b>, <b>अभिमर्शनम्</b> <b>abhimarśanam</b>, <b>अभिमर्षः</b> <

[914, <hw><d><b>निर्णेकः</b> <b>nirṇekaḥ</b></d></hw>]
[919, <hw><d><b>निर्वेदः</b> <b>nirvedaḥ</b></d></hw>]
[938, <hw><d><b>नेत्री</b> <b>netrī</b></d></hw>]
[942, <hw><d><b>न्यञ्चनम्</b> <b>nyañcanam</b></d></hw>]
[944, <hw><d><b>पक्कणः</b> <b>pakkaṇaḥ</b></d></hw>]
[945, <hw><d><b>पक्षकः</b> <b>pakṣakaḥ</b></d></hw>]
[952, <hw><d><b>पटलकः</b> <b>paṭalakaḥ</b>, <b>पटलकम्</b> <b>paṭalakam</b></d></hw>]
[952, <hw><d><b>पटहः</b> <b>paṭahaḥ</b></d></hw>]
[953, <hw><d><b>पट्टिका</b> <b>paṭṭikā</b></d></hw>]
[953, <hw><d><b>पट्टोलिका</b> <b>paṭṭolikā</b></d></hw>]
[955, <hw><d><b>पतङ्गमः</b> <b>pataṅgamaḥ</b></d></hw>]
[955, <hw><d><b>पतङ्गिका</b> <b>pataṅgikā</b></d></hw>]
[968, <hw><d><b>पराञ्जः</b> <b>parāñjaḥ</b></d></hw>]
[978, <hw><d><b>परिधानम्</b> <b>paridhānam</b>, <b>परीधानम्</b> <b>parīdhānam</b></d></hw>]
[981, <hw><d><b>परिभोगः</b> <b>paribhogaḥ</b></d></hw>]
[982, <hw><d><b>परिमरः</b> <b>parimaraḥ</b></d></hw>]
[992, <hw><d><b>पर्दः</b> <b>pardaḥ</b></d></hw>]
[992, <hw><d><

[1466, <hw><d><b>विलोकः</b> <b>vilokaḥ</b>, <b>विलोकनम्</b> <b>vilokanam</b></d></hw>]
[1467, <hw><d><b>विवधः</b> <b>vivadhaḥ</b></d></hw>]
[1467, <hw><d><b>विवधिकः</b> <b>vivadhikaḥ</b></d></hw>]
[1467, <hw><d><b>विवधा</b> <b>vivadhā</b></d></hw>]
[1467, <hw><d><b>विवरम्</b> <b>vivaram</b></d></hw>]
[1469, <hw><d><b>विवर्धनम्</b> <b>vivardhanam</b></d></hw>]
[1479, <hw><d><b>विष्ठा</b> <b>viṣṭhā</b></d></hw>]
[1482, <hw><d><b>विस्फारः</b> <b>visphāraḥ</b></d></hw>]
[1485, <hw><d><b>वीः</b> <b>vīḥ</b></d></hw>]
[1485, <hw><d><b>वीकः</b> <b>vīkaḥ</b></d></hw>]
[1485, <hw><d><b>वीक्षम्</b> <b>vīkṣam</b></d></hw>]
[1485, <hw><d><b>वीक्षणम्</b> <b>vīkṣaṇam</b>, <b>वीक्षणा</b> <b>vīkṣaṇā</b></d></hw>]
[1500, <hw><d><b>वैजयन्तः</b> <b>vaijayantaḥ</b></d></hw>]
[1500, <hw><d><b>वैजयन्तिका</b> <b>vaijayantikā</b></d></hw>]
[1502, <hw><d><b>वैनायिकः</b> <b>vaināyikaḥ</b></d></hw>]
[1504, <hw><d><b>वैशद्यम्</b> <b>vaiśadyam</b></d></hw>]
[1510, <hw><d><b>व्यपायः</b> <b>vyapāyaḥ</b></d></hw>]
[15

In [16]:
# converting the dictionary data into a dataframe
df = pd.DataFrame.from_dict(d)


,word,meaning,pos
2581,अञ्च्,"or 10 U. To manifest, unfold;",verb
2582,अञ्च्,U. (,verb
2583,अञ्च्,"To bend, incline, curl, curve;",verb
2584,अञ्च्,"To go, move, tend towards;",verb
2585,अञ्च्,"To worship, honour, reverence;",verb
2586,अञ्च्,"To request, ask or call for, desire.",verb
2587,अञ्च्,"To murmur, speak indistinctly.",verb
2588,अञ्च् आ,to bend;,verb
2589,अञ्च् उप,to draw or raise (water).,verb
2590,अञ्च् परि,"to cause to revolve, whirl; twist.",verb


In [17]:
# expanding the dictionary d into xd through splitting by semi-colon and commas in the meaning text to get less worded meanings.
xd = {'word':[],'meaning':[],'pos':[]}
def expand_d(row):
    try:
        
        word = row['word']
        meaning = row['meaning']
        pos = row['pos']
        meaning_s1 = meaning.split(';')
        meaning_s2 = []
        for i in meaning_s1:
            s2 = i.split(',')
            for j in s2:
                xd['word'].append(word)
                xd['meaning'].append(j)
                xd['pos'].append(pos)
    except:
        print([word,meaning])



In [18]:
# cleaning the result by removing some unwanted entries
indexNames = df[df['word']=='निर् रस'].index
df = df.drop(indexNames)
indexNames = df[df['meaning']==''].index
df = df.drop(indexNames)
indexNames = df[df['meaning'] == None].index
df = df.drop(indexNames)
df['meaning'] = df.meaning.apply(lambda x:x.strip(' '))


In [19]:
df.apply(lambda row:expand_d(row),axis =1 )

0         None
1         None
2         None
3         None
4         None
          ... 
186435    None
186436    None
186437    None
186438    None
186439    None
Length: 186438, dtype: object

In [20]:
#converting the expanded dictionary to a dataframe
xdf = pd.DataFrame.from_dict(xd)
xdf

,word,meaning,pos
0,अ अः,Tv.],noun
1,अ अः,N. of Viṣṇu,noun
2,अ अः,the first of the three sounds constituting th...,noun
3,अ अः,N. of Śiva,noun
4,अ अः,Brahmā,noun
...,...,...,...
318147,ह्वानम्,A challenge (for fight),noun
318148,ह्वानम्,,noun
318149,ह्वायक,Calling,a.
318150,ह्वायक,summoning,a.


In [21]:
# dropping empty meaning cells from the dataframe
indexNames = xdf[xdf['meaning']==''].index
xdf = xdf.drop(indexNames).reset_index()

In [22]:
#checking the unique parts of speech in the dataframe
xdf.pos.unique()

array(['noun', 'a.', '', 'adv.', 'm.', 'a', 'f.', 'n.', 'ind.', 'm. f.',
       'verb', 'ind', 'm. n.', 'pp.', 'p. p.', 'pot. p.', 'f. n.',
       'm. n.,', 'pron. a.', 'f.,', 'pres. p.', 'p. p', 'a,', 'Caus.',
       'q.', 'm., f.', 'm. pl.', 'pass.', 'pot p.', 'a. pot. p.',
       'Caus. p. p.', 'pot. P.', 'caus.', 'n.,', 'num. a.', 'm., n.',
       'pron.', 'm. a.', '.f', 'Pres. p.', 'a. ', 'pres. a.', 'p.', 'f.;',
       '-ind.', 'Pot. p.', 'a. ind.', '-m.', '-m.,', 'n', 'm.,', 'f. a.',
       'f', 'f., m.', 'i. e.', 'f,', 'ibid.', 'm. (pl.)', 'Onomat.',
       'm., f.,', 'a.,', 'p.p.', 'pron., a.', '-m., f.', '-m., -n.',
       'f., n.', 'a. (pl.)', 'Num. a.', 'Denom.', 'c.', 'a/', 'p. p. ',
       '-Caus.', 'a. (Desid.)', 'm. dual.', 'lapis lazuli', 'p p.', 'm',
       'Lapis lazuli.', 'Pass.', 's. v.', 'Desid.', 'p. p.,', 'Pron. a.',
       'm. du.', 'Agni', 'ind. Ved.'], dtype=object)

In [23]:
# list of the four categories of parts of speech required
pos_l = ['noun','adj','adv','verb']

# dictionary mapping the unique pos values to the above four required
pos_dict = {'a.' : 'adj','f.' : 'noun','m.':'noun','ind.':'adv','n.' : 'noun','m. f.':'noun','pot. p.':'verb','p. p.':'verb',
           'm. n.':'noun','f. n.':'noun','pron. a.':'adj','pres. p.':'verb','a,':'adj','-ind.':'adv','Denom.':'verb','Desid.':'verb',
            'Pot. p.':'verb','a. ind.':'adj','-m.':'noun','p. p.,':'verb','m. a.':'adj','Pron. a.':'adj','-m., f.':'noun',
            '-m., -n.':'noun','Num. a.':'adj','a/':'adj','m. dual.':'noun','p p.':'verb','m':'noun','m. du.':'noun','ind. Ved.':'adv',
            'a':'adj','m.,':'noun','p. p':'verb','m., f.':'noun','m. pl.':'noun','pass.':'verb','pot p.':'verb','a. pot. p.':'verb',
            'pot. P.':'verb','Caus. p. p.':'verb','adv.':'adv','pres. a.':'adj','f.;':'noun','f., n.':'noun','a. (pl.)':'adj',
            'a. (Desid.)':'adj','p.':'noun','n':'noun','f. a.':'adj','f,':'noun','Pass.':'verb','Pres. p.':'verb','f':'noun','num. a.':'adj',
            'f., m.':'noun','pron., a.':'adj','a.,':'adj','-m.,':'noun','m., f.,':'noun','p.p.':'verb','pron.':'noun','.f':'noun',
            'n.,':'noun','m. (pl.)':'noun','adverb':'adv','m., n.':'noun','pp.':'verb','f.,':'noun'
           }

# some methods to clean and refine the meaning and pos data of the dictionary
def handle_1(row):
    return re.sub("[\(\[].*?[\)\]]", "", row).strip(' ')
def handle_2(row):
    return row.split(')')[-1]
def handle_3(row):
    return row.split(']')[-1]
def handle_4(row):
    return row.split('[')[0]
def handle_5(row):
    return row.split('(')[0]
def handle_6(row):
    if 'N. of ' in row:
        return row.replace('N. of ','').strip(' ')
    else:
        return row.strip(' ')
def handle_9(row):
    meaning = row['meaning']
    word = row['word']
    pos = row['pos']
    if 'N.' in meaning or 'n.' in meaning or 'The ' in meaning:
        return 'noun'
    if 'a.' in meaning:
        return 'adj'
    if 'to ' == meaning[:3] or check_verb_pos(meaning) or 'To ' == meaning[:3]:
        return 'verb'
    return pos
def map_pos(row):
    if row not in pos_l and row!='':
        if row.strip(' ') in pos_dict.keys():
            return pos_dict[row.strip(' ')]
        else:
            return row
    else:
        return row
def handle_7(row):
    row = row.replace('\t','')
    row = row.replace('\n','')
    row = row.replace('˚','')
    row = row.strip('"')
    row = row.strip("'")
    row = row.strip('.')
    row = row.strip('*')
    row = row.strip('-')
    row = row.strip('=')
    if len(row)>3:
        if row[:3] == 'or ':
            row =  row[3:]
        if row[:3] == 'An ' or row[:3] == 'an ':
            row =  row[3:]
        if row[:2] == 'A ':
            row = row[2:]
        if row[:2] == 'a ':
            row = row[2:]
        if row[:4] == 'The ' or row[:4] == 'the ':
            row = row[4:]
    
            
    row = row.replace(' a ', '')
    return row
def handle_8(row):
    ret = row.replace('.','')
    if ret.isdigit():
        return ''
    else:
        return row

    

In [24]:
xdf.meaning = xdf.meaning.apply(lambda x : handle_1(x))
xdf.meaning = xdf.meaning.apply(lambda x : handle_2(x))
xdf.meaning = xdf.meaning.apply(lambda x : handle_3(x))
xdf.meaning = xdf.meaning.apply(lambda x : handle_4(x))
xdf.meaning = xdf.meaning.apply(lambda x : handle_5(x))
xdf.pos = xdf.pos.apply(lambda x : map_pos(x))
xdf.pos = xdf.apply(lambda x : handle_9(x),axis=1)
xdf.meaning = xdf.meaning.apply(lambda x : handle_6(x))
xdf.meaning = xdf.meaning.apply(lambda x : handle_7(x))
xdf.meaning = xdf.meaning.apply(lambda x : handle_8(x))

In [25]:
# removing empty meaning cells from the data
indexNames = xdf[xdf['meaning']==''].index
xdf = xdf.drop(indexNames)

In [26]:
#removing unwanted characters from the text
unw = ['. —','a.','adv.','Rv.','Mb.','ind.','N.','n.','f.',' (=','&c.','&c','etc','= ~','(= ~',')','inv.','A.','U.','P.','Ā.','&','[',']','=','.']
def clean_set(m):
    ret = m
    ret = re.sub("[\(\[].*?[\)\]]", "", m)
    for i in unw:
        ret = ret.replace(i,'')
    
    return ret.strip(' ')
    

In [27]:
xdf.meaning = xdf.meaning.apply(lambda x : clean_set(x))

In [196]:
xdf.to_csv('xdf-1.txt',sep='\t')
xdf.to_csv('xdf-1.csv',sep=',')


In [28]:
xdf.pos.unique()

array(['noun', 'adj', 'adv', '', 'verb', 'ind', 'Caus.', 'q.', 'caus.',
       'i. e.', 'ibid.', 'Onomat.', '-Caus.', 'lapis lazuli', 's. v.',
       'Agni'], dtype=object)

In [29]:
# function to remove the verb signifiers (P, A) from the meaning text
def handle_11(row):
    pos = row['pos']
    meaning = row['meaning']
    if pos == 'verb':
        for i in verb_s:
            if i in meaning:
                meaning = meaning.replace(i,'')
    meaning = ''.join([i for i in meaning if not i.isdigit()])
    return meaning.strip(' ')

In [30]:
xdf.meaning = xdf.apply(lambda row:handle_11(row),axis=1)

In [31]:
xdf[xdf.pos.apply(lambda x: x =='')][500:520]

,index,word,meaning,pos
15948,19098,अपवारकः,Covered,
15949,19099,अपवारकः,concealed,
16002,19161,अपवर्गः,Left,
16003,19162,अपवर्गः,abandoned,
16004,19163,अपवर्गः,fulfilled,
16005,19164,अपवर्गः,see above,
16006,19165,अपवर्गः,oft in comp in the sense of 'free from,
16007,19166,अपवर्गः,being without,
16008,19167,अपवर्गः,destitute of,
16009,19168,अपवर्गः,with instr,


In [32]:
xdf[xdf.word.apply(lambda x : x == 'अपव्ययः')]

,index,word,meaning,pos
16099,19279,अपव्ययः,Extravagant expenditure,
16100,19280,अपव्ययः,prodigality,
16101,19281,अपव्ययः,lavishness,


In [35]:
# expanding the data using meanings having the 'or' conjunction.
xxd = {'word':[],'meaning':[],'pos':[]}

def or_split(row):
    meaning = row['meaning']
    pos = row['pos']
    word = row['word']
    split_sp = meaning.split(' ')
    if len(split_sp) == 3 and split_sp[1]=='or':
        xxd['word'].append(word)
        xxd['meaning'].append(split_sp[0])
        xxd['pos'].append(pos)
        xxd['word'].append(word)
        xxd['meaning'].append(split_sp[2])
        xxd['pos'].append(pos)
    elif len(split_sp) ==4 and split_sp[0] == 'To' and split_sp[2] == 'or' and pos == 'verb':
        xxd['word'].append(word)
        xxd['meaning'].append(split_sp[1])
        xxd['pos'].append(pos)
        xxd['word'].append(word)
        xxd['meaning'].append(split_sp[3])
        xxd['pos'].append(pos)
    else:
        xxd['word'].append(word)
        xxd['meaning'].append(meaning.strip(' '))
        xxd['pos'].append(pos)

        

In [49]:
xdf.apply(lambda row: or_split(row),axis=1)
xxdf = pd.DataFrame.from_dict(xxd)
xxdf

,word,meaning,pos
0,अ अः,Viṣṇu,noun
1,अ अः,first of the three sounds constituting the sac...,noun
2,अ अः,Śiva,noun
3,अ अः,Brahmā,noun
4,अ अः,Vāyu,noun
...,...,...,...
499757,ह्वानम्,sound,noun
499758,ह्वानम्,challenge,noun
499759,ह्वायक,Calling,adj
499760,ह्वायक,summoning,adj


In [51]:
suff = ['ा','ः','म','ी']
def handle_double_word(row):
    #word = row['Sans_Label']
    word = row['word']
    if len(word.split(' ')) == 2:
        try:
            fword = word.split(' ')[0]
            sword = word.split(' ')[1]
            fword = fword[:-2] if fword[-2:] == 'म्' else fword
            fword = fword[:-1] if fword[-1] == '्' else fword
            fword = fword[:-1] if fword[-1] == 'ः' else fword
            sword = sword[:-1] if sword[-1] == '्' else sword
            sword = sword[:-1] if sword[-1] == 'ः' else sword
            sword = sword[:-1] if sword[-1] in suff else sword
            if sword == '':
                return 1
            elif check_suff(sword,fword):
                return 1
            else:
                return 0
            return 0
        except:
            print([word,fword,sword])
            return 0
    return 0


In [52]:
xxdf.word = xxdf.word.apply(lambda x:x.strip(' '))
xxdf['dub'] = xxdf.apply(lambda row: handle_double_word(row),axis=1)

In [53]:
def remove_doub(row):
    if row['dub'] ==1:
        word = row['word'].split(' ')[0]
        return word
    return row['word']

In [54]:
xxdf[xxdf.dub==1]
xxdf.word = xxdf.apply(lambda row:remove_doub(row),axis=1)

In [55]:
def rem_suff(row):
    #word = row['Sans_Label']
    word = row['word']
    if word[-1] == 'ः' or word[-1] ==':':
        word= word[:-1]
    if word[-2:] == 'म्':
        word = word[:-2]
    if ' ' in word:
        sp_split = word.split(' ')
        if sp_split[0][-1] == 'ः' or sp_split[0][-1] ==':':
            word = sp_split[0][:-1] + ' ' + sp_split[1]
        elif sp_split[0][-2:] == 'म्':
            word = sp_split[0][:-2] + ' ' +sp_split[1]
    return word

In [56]:
#df2['Sans_Label'] = df2.apply(lambda x: rem_suff(x),axis=1)
xxdf['word'] = xxdf.apply(lambda x: rem_suff(x),axis=1)

In [57]:
xxdf

,word,meaning,pos,dub
0,अ,Viṣṇu,noun,1
1,अ,first of the three sounds constituting the sac...,noun,1
2,अ,Śiva,noun,1
3,अ,Brahmā,noun,1
4,अ,Vāyu,noun,1
...,...,...,...,...
499757,ह्वान,sound,noun,0
499758,ह्वान,challenge,noun,0
499759,ह्वायक,Calling,adj,0
499760,ह्वायक,summoning,adj,0


df2[df2.Sans_Label.apply(lambda x : 'म्' in x)][530:550]

In [59]:
xxdf['pos'] = xxdf.apply(lambda x: handle_13(x),axis =1)

In [60]:
#xxdf[xxdf['meaning'] == 'Calling']
def handle_14(row):
    pos = row['pos']
    word = row['word']
    if pos == '' and spos_dict2[word] != '':
        return spos_dict2[word]
    elif pos =='' and spos_dict2[word] == '':
        return 'noun'
    return pos

In [61]:
xxdf['pos'] = xxdf.apply(lambda row:handle_14(row),axis=1)

In [609]:
sxxdf = xxdf[xxdf.meaning.apply(lambda x : ' ' not in x)]
sxxdf

,word,meaning,pos,dub
0,अ,Viṣṇu,noun,1
2,अ,Śiva,noun,1
3,अ,Brahmā,noun,1
4,अ,Vāyu,noun,1
5,अ,Vaiśvānar,adj,1
...,...,...,...,...
249876,ह्वान,sound,noun,0
249877,ह्वान,challenge,noun,0
249878,ह्वायक,Calling,adj,0
249879,ह्वायक,summoning,adj,0


In [ ]:
def handle_13(row):
    pos = row['pos']
    word = row['word']
    if word not in spos_dict2.keys() and word != '':
        spos_dict2[word] = pos
        return pos
    elif word in spos_dict2.keys() and spos_dict2[word] != pos and pos != '' and spos_dict2[word] != '':
        disp_list2.append([word,pos,spos_dict2[word]])
        return spos_dict2[word]
    elif word in spos_dict2.keys() and  spos_dict2[word] != pos and pos == '':
        disp_list2.append([word,pos,spos_dict2[word]])
        return spos_dict2[word]
    elif word in spos_dict2.keys() and  spos_dict2[word] != pos and spos_dict2[word] =='':
        spos_dict2[word] = pos
        return pos
    return pos


In [349]:
sxxdf.pos.unique()

array(['noun', 'adj', 'adv', 'verb', 'ind', 'Caus.', 'q.', 'caus.',
       'i. e.', 'ibid.', '-Caus.', 's. v.', 'Agni'], dtype=object)

In [63]:
def handle_16(row):
    pos = row['pos']
    meaning = row['meaning']
    if 'caus' in pos.lower():
        return 'verb'
    elif pos == 'ind' and meaning[-2:] == 'ly':
        return 'adv'
    elif pos not in pos_l:
        return 'noun'
    return pos
        
    

In [64]:
#sxxdf['pos'] = sxxdf.apply(lambda row:handle_16(row),axis=1)
xxdf['pos'] = xxdf.apply(lambda row:handle_16(row),axis=1)

In [66]:
#sxxdf.pos.unique()
xxdf.pos.unique()

array(['noun', 'adj', 'adv', 'verb'], dtype=object)

In [67]:
#indexNames = sxxdf[sxxdf['meaning']==''].index
#sxxdf = sxxdf.drop(indexNames)
#indexNames = sxxdf[sxxdf['meaning']=='cf'].index
#sxxdf = sxxdf.drop(indexNames)
indexNames = xxdf[xxdf['meaning']==''].index
xxdf = xxdf.drop(indexNames)
indexNames = xxdf[xxdf['meaning']=='cf'].index
xxdf = xxdf.drop(indexNames)

In [68]:
def check_digit(row):
    if row.isdigit():
        return True
    return False

In [69]:
#df2[df2.Sans_Label.apply(lambda x:check_digit(x))]
xxdf[xxdf.word.apply(lambda x:check_digit(x))]


,word,meaning,pos,dub


In [70]:
#sxxdf = sxxdf.reset_index()
xxdf = xxdf.reset_index()

In [72]:
#sxxdf = sxxdf.drop(['level_0'],axis=1)
xxdf 

,index,word,meaning,pos,dub
0,0,अ,Viṣṇu,noun,1
1,1,अ,first of the three sounds constituting the sac...,noun,1
2,2,अ,Śiva,noun,1
3,3,अ,Brahmā,noun,1
4,4,अ,Vāyu,noun,1
...,...,...,...,...,...
497049,499757,ह्वान,sound,noun,0
497050,499758,ह्वान,challenge,noun,0
497051,499759,ह्वायक,Calling,adj,0
497052,499760,ह्वायक,summoning,adj,0


In [75]:
#duplicate = sxxdf[sxxdf.duplicated(['word', 'meaning'])]
#duplicate
#sxxdf = sxxdf.drop_duplicates(subset=['word','meaning'], keep='first').reset_index()
#sxxdf
duplicate = xxdf[xxdf.duplicated(['word', 'meaning'])]
duplicate
xxdf = xxdf.drop_duplicates(subset=['word','meaning'], keep='first').reset_index()
xxdf

,level_0,index,word,meaning,pos,dub
0,0,0,अ,Viṣṇu,noun,1
1,1,1,अ,first of the three sounds constituting the sac...,noun,1
2,2,2,अ,Śiva,noun,1
3,3,3,अ,Brahmā,noun,1
4,4,4,अ,Vāyu,noun,1
...,...,...,...,...,...,...
241977,248522,249876,ह्वान,sound,noun,0
241978,248523,249877,ह्वान,challenge,noun,0
241979,248524,249878,ह्वायक,Calling,adj,0
241980,248525,249879,ह्वायक,summoning,adj,0


In [76]:
#sxxdf  = sxxdf.drop(['index','level_0'],axis=1)
xxdf  = xxdf.drop(['index','level_0'],axis=1)

In [77]:
def check_length(row):
    if len(row)==2:
        return True
    return False

In [78]:
#sxxdf[sxxdf.meaning.apply(lambda x:check_length(x))].meaning.unique()
xxdf[xxdf.meaning.apply(lambda x:check_length(x))].meaning.unique()

array(['Mb', 'of', 'so', 'pe', 'Mu', 'Ts', 'an', 'go', 'or', 'IV', 'VI',
       'as', 'su', 'Pt', 'If', 'Av', 'bl', 'Si', 'ox', 'to', 'at', 'Ki',
       'Śi', 'Śr', 'In', 'II', 'Mv', 'No', 'Me', 'Ms', 'To', 'Ve', 'Dk',
       'Uṇ', 'it', 'Bk', 'do', 'Ku', 'Mk', 'oh', 'ah', 'nd', 'Su', 'ma',
       'Ks', 'Bh', 'Pr', 'th', 'Bg', 'be', 'Bv', 'is', 'fh', 'Ti', 'EI',
       'MS', 'V-', 'up', 'on', 'On', 'Vb', 'Ch', 'in', 'fu', 'Or', 'st',
       'An', 'he', 'fe', 'by', 'AV', 'MW', 'Ṛs', 'vh', 'Tv', 'pi', 'Nm',
       'So', 'He', 'R-', 'YI', 'Om', 'Oh', 'Cf', 'fa', 'M/', 'BR', 'ff',
       'bo', 'XX', 'ru', 'se', 'My', 'si', 'As', 'if', 'pl', 'gr', 'Rv',
       'Ṛv', 'ŚB', 'eg', 'du', 'ho'], dtype=object)

In [79]:
unw_two = ['Mb', 'pe', 'Mu', 'Ts','IV', 'VI',
       'as', 'Pt', 'Av', 'bl', 'Si', 'Ki',
       'Śi', 'Śr', 'II', 'Mv', 'Ms', 'Dk',
       'Uṇ', 'Bk', 'Ku', 'Mk', 'Su', 'ma',
       'Ks', 'Bh', 'Pr', 'th', 'Bg', 'Bv', 'fh', 'Ti', 'EI',
       'MS', 'V-', 'Vb', 'Ch',  'fu',  'st',
       'fe', 'AV', 'MW', 'Ṛs', 'vh', 'Tv', 'pi', 'Nm',
     'R-', 'YI', 'Cf', 'fa', 'M/', 'BR', 'ff',
       'bo', 'XX', 'ru', 'se', 'si', 'pl', 'gr', 'Rv',
       'Ṛv', 'ŚB', 'eg', 'du', 'ho']
def rem_unw(row):
    if len(row) == 1:
        return 1
    elif row in unw_two:
        return 1
    return 0


In [80]:
#sxxdf['unw'] = sxxdf.meaning.apply(lambda x:rem_unw(x))
xxdf['unw'] = xxdf.meaning.apply(lambda x:rem_unw(x))

In [82]:
xxdf[xxdf.unw==1]

,word,meaning,pos,dub,unw
100,अंस्,L,noun,0,1
320,अकाण्ड पातजात,H,adj,0,1
509,अकृत व्रण,Mb,adj,0,1
893,अक्षर छन्दस्,n,noun,0,1
896,अक्षर जननी,pe,noun,0,1
...,...,...,...,...,...
241486,हेमन् अङ्क,Mu,adj,0,1
241515,हेमन् कूट,Ś,adj,0,1
241724,होम,R,noun,0,1
241793,ह्यस्त्य,IV,verb,0,1


In [83]:
xxdf = xxdf[xxdf.unw ==0]

In [84]:
#sxxdf = sxxdf.reset_index()
#sxxdf = sxxdf.drop(['index'],axis=1)
xxdf = xxdf.reset_index()
xxdf = xxdf.drop(['index'],axis=1)

In [85]:
#sxxdf = sxxdf.drop(['dub','unw'],axis=1)
xxdf = xxdf.drop(['dub','unw'],axis=1)

In [449]:
s='ह्वानम्'
s[:-1][-1]

'म'

In [48]:
def check_suff(s, t):
    s_len = len(s)
    t_len = len(t)
    if (s_len > t_len):
        return False
    for i in range(s_len):
        if(s[s_len - i - 1] != t[t_len - i - 1]):
            return False
    return True

In [44]:
df2[df2.Sans_Label.apply(lambda x: 'ः' in  x)]

,Sans_Label,Eng_Label,POS,MRS_Concept,MRS_Feature_Values,dub
624,अक्षि कूटः-टक,eyeball,noun,_eyeball_n_1,LBL: h* ARG0: x*,0
2128,अट्ट ट्टः-ट्ट,garret,adj,_garret/JJ_u_unknown,LBL: h* ARG0: e* ARG1: x*,0
2133,अट्ट ट्टः-ट्ट,Killing,adj,named,",CARG:_VALUES",0
2135,अट्ट ट्टः-ट्ट,injuring,adj,_injure_v_1,LBL: h* ARG0: e* ARG1: i* ARG2: p*,0
2136,अट्ट ट्टः-ट्ट,injuring,adj,nominalization,LBL: h* ARG0: x* ARG1: h*,0
...,...,...,...,...,...,...
137477,वासस् वासःकुटी,tent,noun,_tent_n_1,LBL: h* ARG0: x*,0
156264,श्वस् श्वःश्रेयस,happy,adj,_happy_a_with,LBL: h* ARG0: e* ARG1: x* ARG2: i*,0
156265,श्वस् श्वःश्रेयस,prosperous,adj,_prosperous_a_1,LBL: h* ARG0: e* ARG1: x*,0
161638,सम दुःख,sympathising,adj,_sympathize_v_with,LBL: h* ARG0: e* ARG1: x*,0


In [94]:
#sxxdf[sxxdf.meaning=='see']
xxdf[xxdf.meaning=='see']

,word,meaning,pos
1584,अग्नि ष्टुत्,see,adj
1585,अग्नि ष्टुभ,see,adj
1586,अग्नि ष्टोम,see,adj
1590,अग्नि ष्वात्त,see,adj
3956,अतिपत्,see,verb
...,...,...,...
237200,हलाहल,see,noun
237765,हास्य रस,see,adj
238159,हिरण्य कशिपु,see,adj
238288,हीही,see,adv


In [92]:
def rem_see(row):
    pos = row['pos']
    meaning = row['word']
    if pos != 'verb' and meaning == 'see':
        meaning = ''
    return meaning

In [93]:
xxdf['word'] = xxdf.apply(lambda x:rem_see(x),axis=1)

In [47]:
df2['Eng_Label'] = df2.apply(lambda x:rem_see(x),axis=1)

C:\Users\aparashar\AppData\Local\Temp\ipykernel_7928\3623128876.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Eng_Label'] = df2.apply(lambda x:rem_see(x),axis=1)


In [48]:
indexNames = df2[df2['Eng_Label']==''].index
df2 = df2.drop(indexNames).reset_index()

In [96]:
indexNames = xxdf[xxdf['word']==''].index
xxdf = xxdf.drop(indexNames).reset_index()

In [106]:
def rem_verb_pref(row):
    pos = row['pos']
    meaning = row['meaning']
    if pos =='verb' and meaning[:3].lower() == 'to ':
        return meaning[3:]
    return meaning
xxdf['meaning'] = xxdf.apply(lambda x:rem_verb_pref(x),axis=1)

In [113]:
#xxdf= xxdf.drop(['index'],axis=1)
def isCompound(row):
    word = row['word']
    meaning = row['meaning']
    if ' ' in word or ' ' in meaning:
        return 1
    return 0
xxdf['comp'] = xxdf.apply(lambda x:isCompound(x),axis=1)
cxxdf = xxdf[xxdf.comp ==1]

In [115]:
cxxdf = cxxdf.reset_index()

In [118]:
#sxxdf.to_csv('apte_sanskrit-eng-v2.txt',sep='\t')
cxxdf.to_csv('apte_sanskrit-eng-compound-v1.txt',sep='\t')
cxxdf.to_csv('apte_sanskrit-eng-compound-v1.csv',sep=',')


In [117]:
cxxdf = cxxdf.drop(['index','comp'],axis=1)

In [12]:
df1 = pd.read_csv('apte_sanskrit-eng-v2.txt',sep='\t')
df2 = pd.read_csv('ace-0.9.24//apte_sanskrit-eng_v1_dict.txt',sep='\t')

In [22]:
df2
df2[df2.Eng_Label.apply(lambda x:check_length(x))].Eng_Label.unique()
df2[df2.Eng_Label=='see']

,Sans_Label,Eng_Label,POS,MRS_Concept,MRS_Feature_Values
2550,अतिपत्,see,verb,_see_v_1,LBL: h* ARG0: e* ARG1: x*
5624,अनुकृष्,see,verb,_see_v_1,LBL: h* ARG0: e* ARG1: x*
5934,अनुत्पादः क्षान्ति,see,noun,_see_n_1,LBL: h* ARG0: x*
6124,अनुपद्,see,verb,_see_v_1,LBL: h* ARG0: e* ARG1: x*
6431,अनुमा,see,verb,_see_v_1,LBL: h* ARG0: e* ARG1: x*
...,...,...,...,...,...
175197,हलाहलः,see,noun,_see_n_1,LBL: h* ARG0: x*
175199,हलाहलम्,see,noun,_see_n_1,LBL: h* ARG0: x*
176017,हीही,see,adv,pron,LBL: h* ARG0: x*
176018,हीही,see,adv,_see_v_1,LBL: h* ARG0: e* ARG1: x* ARG2: h*


In [14]:
unw_concs = ["compound","number_q","udef_q","pronoun_q","proper_q","person","parg_d","generic_entity","numbered_hour",
 "person","def_implicit_q","loc_nonsp","time_n","superl","def_explicit_q","poss","comp"]
df2  = df2[df2.MRS_Concept.apply(lambda x:x not in unw_concs)]

In [51]:
df2 = df2.drop(['index','dub'],axis=1)

In [24]:
df1[df1.meaning=='see']

,Unnamed: 0,word,meaning,pos
1908,1908,अतिपत्,see,verb
4252,4252,अनुकृष्,see,verb
4620,4620,अनुपद्,see,verb
4864,4864,अनुमा,see,verb
5120,5120,अनुविद्,see,verb
...,...,...,...,...
127132,127132,सिक्त,see,verb
128189,128189,सुप्त,see,verb
128960,128960,सृष्ट,see,verb
131752,131752,स्रै,see,verb


In [55]:
df2[df2.Sans_Label.apply(lambda x : 'Oops!' not in x)]

,Sans_Label,Eng_Label,POS,MRS_Concept,MRS_Feature_Values
2,अ,Viṣṇu,noun,named,",CARG:_VALUES"
3,अ,Śiva,noun,named,",CARG:_VALUES"
4,अ,Vāyu,noun,named,",CARG:_VALUES"
5,अ,Vaiśvānar,adj,named,",CARG:_VALUES"
6,अंश,share,noun,_share_n_of,LBL: h* ARG0: x*
...,...,...,...,...,...
130944,ह्वान,challenge,noun,_challenge_n_1,LBL: h* ARG0: x*
130945,ह्वायक,Calling,adj,named,",CARG:_VALUES"
130946,ह्वायक,summoning,adj,_summon_v_to,LBL: h* ARG0: e* ARG1: i* ARG2: p* ARG3: i*
130947,ह्वायक,summoning,adj,nominalization,LBL: h* ARG0: x* ARG1: h*


In [60]:

# adding count at the end of the words and meanings
mrs_word_cnt = {}
mrs_mean_cnt = {}
def count_word(row):
    word = row['Sans_Label']
    if word in mrs_word_cnt.keys():
        mrs_word_cnt[word] +=1
        return word+'_'+ str(mrs_word_cnt[word])
    else:
        mrs_word_cnt[word] =1
        return word+'_'+ str(mrs_word_cnt[word])
def count_meaning(row):
    meaning = row['Eng_Label']
    if meaning in mrs_mean_cnt.keys():
        mrs_mean_cnt[meaning] +=1
        return meaning+'_'+ str(mrs_mean_cnt[meaning])
    else:
        mrs_mean_cnt[meaning] =1
        return meaning+'_'+ str(mrs_mean_cnt[meaning])
        

In [61]:
ndf2 = df2
ndf2['Sans_Label'] = ndf2.apply(lambda row:count_word(row),axis =1)
ndf2['Eng_Label'] = ndf2.apply(lambda row:count_meaning(row),axis =1)

ndf2

C:\Users\aparashar\AppData\Local\Temp\ipykernel_7928\2183572935.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf2['Sans_Label'] = ndf2.apply(lambda row:count_word(row),axis =1)
C:\Users\aparashar\AppData\Local\Temp\ipykernel_7928\2183572935.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf2['Eng_Label'] = ndf2.apply(lambda row:count_meaning(row),axis =1)


,Sans_Label,Eng_Label,POS,MRS_Concept,MRS_Feature_Values
2,अ_1,Viṣṇu_1,noun,named,",CARG:_VALUES"
3,अ_2,Śiva_1,noun,named,",CARG:_VALUES"
4,अ_3,Vāyu_1,noun,named,",CARG:_VALUES"
5,अ_4,Vaiśvānar_1,adj,named,",CARG:_VALUES"
6,अंश_1,share_1,noun,_share_n_of,LBL: h* ARG0: x*
...,...,...,...,...,...
130944,ह्वान_8,challenge_14,noun,_challenge_n_1,LBL: h* ARG0: x*
130945,ह्वायक_1,Calling_20,adj,named,",CARG:_VALUES"
130946,ह्वायक_2,summoning_5,adj,_summon_v_to,LBL: h* ARG0: e* ARG1: i* ARG2: p* ARG3: i*
130947,ह्वायक_3,summoning_6,adj,nominalization,LBL: h* ARG0: x* ARG1: h*


In [25]:
def get_mrs_features(row):
    meaning = row['meaning']
    pos = row['pos']
    condit1 = df2[df2['Eng_Label'] == meaning]
    cross = condit1[condit1['POS'] == pos]
    if cross.empty == True:
        #print([meaning,pos])
        return 'OOps'
    else:
        if len(cross)>1:
            #print(['ooo',meaning,pos])
            ret = cross['MRS_Concept'].values[0] + '&&&'+ cross['MRS_Feature_Values'].values[0]
            uq_mrs = np.unique(cross['MRS_Concept'].values)
            if len(uq_mrs) > 1:
                print([meaning,pos,uq_mrs])
        else:
            ret = cross['MRS_Concept'].values[0] + '&&&'+ cross['MRS_Feature_Values'].values[0]
        return ret

In [26]:
df1['mrs_concept'] = df1.apply(lambda x :get_mrs_features(x),axis=1 )

['tree', 'noun', array(['_line_n_of', '_tree_n_of'], dtype=object)]
['unengaged', 'adj', array(['_engage_v_1', '_un-_a_rvrs'], dtype=object)]
['unfettered', 'adj', array(['_fetter_v_1', '_un-_a_rvrs'], dtype=object)]
['unexpected', 'adj', array(['_expect_v_1', 'neg'], dtype=object)]
['undiminished', 'adj', array(['_diminish_v_cause', '_un-_a_rvrs'], dtype=object)]
['unconquered', 'adj', array(['_conquer_v_1', '_un-_a_rvrs'], dtype=object)]
['good-for-nothing', 'adj', array(['_no_q', 'thing'], dtype=object)]
['untroubled', 'adj', array(['_trouble_v_1', '_un-_a_rvrs'], dtype=object)]
['gambling', 'noun', array(['_gamble_v_1', 'nominalization'], dtype=object)]
['tree', 'noun', array(['_line_n_of', '_tree_n_of'], dtype=object)]
['non-forbearing', 'adj', array(['_forbear_v_1', 'nominalization'], dtype=object)]
['undying', 'adj', array(['_die_v_1', '_un-_a_rvrs'], dtype=object)]
['undecaying', 'adj', array(['_decay_v_1', '_un-_a_rvrs'], dtype=object)]
['unlettered', 'adj', array(['_letter_v_

['obtaining', 'noun', array(['_obtain_v_hold', 'nominalization'], dtype=object)]
['looking', 'noun', array(['_look_v_1', 'nominalization'], dtype=object)]
['obtaining', 'noun', array(['_obtain_v_hold', 'nominalization'], dtype=object)]
['looking', 'noun', array(['_look_v_1', 'nominalization'], dtype=object)]
['warming', 'noun', array(['_warm_v_1', 'nominalization'], dtype=object)]
['boiling', 'noun', array(['_boil_v_1', 'nominalization'], dtype=object)]
['warming', 'noun', array(['_warm_v_1', 'nominalization'], dtype=object)]
['boiling', 'noun', array(['_boil_v_1', 'nominalization'], dtype=object)]
['guiding', 'noun', array(['_guide_v_to', 'nominalization'], dtype=object)]
['governing', 'noun', array(['_govern_v_1', 'nominalization'], dtype=object)]
['occupying', 'noun', array(['_occupy_v_1', 'nominalization'], dtype=object)]
['inhabiting', 'noun', array(['_inhabit_v_1', 'nominalization'], dtype=object)]
['instructing', 'adj', array(['_instruct_v_on', 'nominalization'], dtype=object)]


['mentioning', 'noun', array(['_mention_v_to', 'nominalization'], dtype=object)]
['mentioning', 'noun', array(['_mention_v_to', 'nominalization'], dtype=object)]
['doing', 'noun', array(['_do_v_1', 'nominalization'], dtype=object)]
['pursuing', 'noun', array(['_pursue_v_1', 'nominalization'], dtype=object)]
['pursuing', 'noun', array(['_pursue_v_1', 'nominalization'], dtype=object)]
['scrutinizing', 'adj', array(['_scrutinize_v_1', 'nominalization'], dtype=object)]
['scrutinizing', 'adj', array(['_scrutinize_v_1', 'nominalization'], dtype=object)]
['remembering', 'noun', array(['_remember_v_from', 'nominalization'], dtype=object)]
['Underneath', 'adj', array(['_underneath_p', 'place_n'], dtype=object)]
['inside', 'adj', array(['_inside_p', 'place_n'], dtype=object)]
['destroying', 'adj', array(['_destroy_v_1', 'nominalization'], dtype=object)]
['Inside', 'adv', array(['_inside_p', 'place_n'], dtype=object)]
['Inside', 'adv', array(['_inside_p', 'place_n'], dtype=object)]
['near', 'adv'

['removing', 'noun', array(['_remove_v_1', 'nominalization'], dtype=object)]
['dissembling', 'noun', array(['_dissembling/VBG_u_unknown', 'nominalization'], dtype=object)]
['obtaining', 'noun', array(['_obtain_v_hold', 'nominalization'], dtype=object)]
['hiding', 'noun', array(['_hide_v_away', 'nominalization'], dtype=object)]
['satisfying', 'noun', array(['_satisfy_v_1', 'nominalization'], dtype=object)]
['undigested', 'adj', array(['_digest_v_1', '_un-_a_rvrs'], dtype=object)]
['removing', 'adj', array(['_remove_v_1', 'nominalization'], dtype=object)]
['eclipsing', 'adj', array(['_eclipse_v_1', 'nominalization'], dtype=object)]
['Westward', 'adv', array(['_westward_a_1', 'place_n'], dtype=object)]
['southward', 'adv', array(['_southward_a_1', 'place_n'], dtype=object)]
['Southern', 'adv', array(['_southern_a_1', 'subord'], dtype=object)]
['Westward', 'adv', array(['_westward_a_1', 'place_n'], dtype=object)]
['southward', 'adv', array(['_southward_a_1', 'place_n'], dtype=object)]
['un

['approaching', 'noun', array(['_approach_v_1', 'nominalization'], dtype=object)]
['assailing', 'noun', array(['_assail_v_1', 'nominalization'], dtype=object)]
['assailing', 'adj', array(['_assail_v_1', 'nominalization'], dtype=object)]
['encountering', 'adj', array(['_encounter_v_1', 'nominalization'], dtype=object)]
['assailing', 'adj', array(['_assail_v_1', 'nominalization'], dtype=object)]
['encountering', 'adj', array(['_encounter_v_1', 'nominalization'], dtype=object)]
['accusing', 'adj', array(['_accuse_v_of', 'nominalization'], dtype=object)]
['pretender', 'adj', array(['_pre-_a_ante', '_tender_n_1'], dtype=object)]
['assaulting', 'adj', array(['_assault_v_1', 'nominalization'], dtype=object)]
['preserve', 'verb', array(['_pre-_a_ante', '_serve_n_1'], dtype=object)]
['reaping', 'noun', array(['_ape_v_1', '_re-_a_again', 'nominalization'], dtype=object)]
['mowing', 'noun', array(['_mow_v_1', 'nominalization'], dtype=object)]
['inscribing', 'adj', array(['_inscribe_v_1', 'nominal

...

['tree', 'noun', array(['_line_n_of', '_tree_n_of'], dtype=object)]
['blessing', 'adj', array(['_bless_v_1', 'nominalization'], dtype=object)]
['extolling', 'noun', array(['_extol_v_1', 'nominalization'], dtype=object)]
['breathing', 'noun', array(['_breathe_v_1', 'nominalization'], dtype=object)]
['Sighing', 'noun', array(['_sigh_v_to', 'nominalization'], dtype=object)]
['heaving', 'noun', array(['_heave_v_2', 'nominalization'], dtype=object)]
['controlling', 'adj', array(['_control_v_1', 'nominalization'], dtype=object)]
['proceeding', 'noun', array(['_proceed_v_1', 'nominalization'], dtype=object)]
['driving', 'noun', array(['_drive_v_1', 'nominalization'], dtype=object)]
['prevent', 'verb', array(['_pre-_a_ante', '_vent_n_1'], dtype=object)]
['prevent', 'verb', array(['_pre-_a_ante', '_vent_n_1'], dtype=object)]
['stopping', 'noun', array(['_stop_v_1', 'nominalization'], dtype=object)]
['confining', 'noun', array(['_confine_v_to', 'nominalization'], dtype=object)]
['Healing', 'noun

['existing', 'noun', array(['_exist_v_1', 'nominalization'], dtype=object)]
['cheating', 'noun', array(['_cheat_v_1', 'nominalization'], dtype=object)]
['decaying', 'noun', array(['_decay_v_1', 'nominalization'], dtype=object)]
['perishing', 'noun', array(['_perish_v_1', 'nominalization'], dtype=object)]
['neighbouring', 'adj', array(['_neighbor_v_1', 'nominalization'], dtype=object)]
['possessing', 'adj', array(['_possess_v_1', 'nominalization'], dtype=object)]
['Obtaining', 'noun', array(['_obtain_v_hold', 'nominalization'], dtype=object)]
['extending', 'noun', array(['_extend_v_1', 'nominalization'], dtype=object)]
['inflaming', 'noun', array(['_inflame_v_1', 'nominalization'], dtype=object)]
['inflaming', 'noun', array(['_inflame_v_1', 'nominalization'], dtype=object)]
['afflicting', 'noun', array(['_afflict_v_1', 'nominalization'], dtype=object)]
['delighting', 'noun', array(['_delight_v_1', 'nominalization'], dtype=object)]
['dividing', 'noun', array(['_divide_v_1', 'nominalizati

['concerning', 'adj', array(['_concern_v_1', 'nominalization'], dtype=object)]
['obstructing', 'noun', array(['_obstruct_v_1', 'nominalization'], dtype=object)]
['addressing', 'noun', array(['_address_v_to', 'nominalization'], dtype=object)]
['instructing', 'noun', array(['_instruct_v_on', 'nominalization'], dtype=object)]
['informing', 'noun', array(['_inform_v_1', 'nominalization'], dtype=object)]
['throwing', 'noun', array(['_throw_v_1', 'nominalization'], dtype=object)]
['possessing', 'noun', array(['_possess_v_1', 'nominalization'], dtype=object)]
['honouring', 'noun', array(['_honor_v_1', 'nominalization'], dtype=object)]
['splitting', 'noun', array(['_split_v_1', 'nominalization'], dtype=object)]
['Blossoming', 'noun', array(['_blossom_v_1', 'nominalization'], dtype=object)]
['using', 'adj', array(['_use_v_1', 'nominalization'], dtype=object)]
['arising', 'noun', array(['_arise_v_1', 'nominalization'], dtype=object)]
['reflecting', 'noun', array(['_reflect_v_1', 'nominalization'

['moving', 'noun', array(['_move_v_1', 'nominalization'], dtype=object)]
['Sprinkling', 'noun', array(['_sprinkle_v_1', 'nominalization'], dtype=object)]
['watering', 'noun', array(['_water_v_1', 'nominalization'], dtype=object)]
['Sprinkling', 'noun', array(['_sprinkle_v_1', 'nominalization'], dtype=object)]
['Sprinkling', 'noun', array(['_sprinkle_v_1', 'nominalization'], dtype=object)]
['watering', 'noun', array(['_water_v_1', 'nominalization'], dtype=object)]
['dripping', 'noun', array(['_drip_v_1', 'nominalization'], dtype=object)]
['reaching', 'noun', array(['_reach_v_1', 'nominalization'], dtype=object)]
['honouring', 'adj', array(['_honor_v_1', 'nominalization'], dtype=object)]
['practising', 'noun', array(['_practice_v_1', 'nominalization'], dtype=object)]
['employing', 'noun', array(['_employ_v_1', 'nominalization'], dtype=object)]
['enjoying', 'noun', array(['_enjoy_v_1', 'nominalization'], dtype=object)]
['fastening', 'noun', array(['_fasten_v_1', 'nominalization'], dtype=o

['charming', 'noun', array(['_charm_v_1', 'nominalization'], dtype=object)]
['sleeping', 'noun', array(['_sleep_v_1', 'nominalization'], dtype=object)]
['accepting', 'noun', array(['_accept_v_1', 'nominalization'], dtype=object)]
['agreeing', 'noun', array(['_agree_v_on', 'nominalization'], dtype=object)]
['promising', 'noun', array(['_promise_v_1', 'nominalization'], dtype=object)]
['accepting', 'noun', array(['_accept_v_1', 'nominalization'], dtype=object)]
['agreeing', 'noun', array(['_agree_v_on', 'nominalization'], dtype=object)]
['promising', 'noun', array(['_promise_v_1', 'nominalization'], dtype=object)]
['accepting', 'noun', array(['_accept_v_1', 'nominalization'], dtype=object)]
['agreeing', 'noun', array(['_agree_v_on', 'nominalization'], dtype=object)]
['promising', 'noun', array(['_promise_v_1', 'nominalization'], dtype=object)]
['uncontrolled', 'adj', array(['_control_v_1', '_un-_a_rvrs'], dtype=object)]
['unreserved', 'adj', array(['_reserve_v_1', '_un-_a_rvrs'], dtype=o

In [62]:
ndf2.to_csv('apte_sanskrit-eng-v3.txt',sep='\t')
ndf2.to_excel('apte_sanskrit-eng-v3.xlsx')